# Lesson 2: Interacting with a CSV Data

## Setup and connect to the Azure OpenAI endpoint

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

![](../images/20-db-agent.png)

In [25]:
import os 
from dotenv import load_dotenv
import pandas as pd
from langchain_openai import AzureChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [26]:
load_dotenv()

True

In [27]:
model = AzureChatOpenAI(
    openai_api_version=os.getenv('OPENAI_API_VERSION'),
    azure_deployment=os.getenv('AZURE_DEPLOYMENT'),
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_ENDPOINT')
)

## Load the dataset

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [30]:
df = pd.read_csv("./data/all-states-history.csv").fillna(value = 0)

             date state    death  deathConfirmed  deathIncrease  \
0      2021-03-07    AK    305.0             0.0              0   
1      2021-03-07    AL  10148.0          7963.0             -1   
2      2021-03-07    AR   5319.0          4308.0             22   
3      2021-03-07    AS      0.0             0.0              0   
4      2021-03-07    AZ  16328.0         14403.0              5   
...           ...   ...      ...             ...            ...   
20775  2020-01-17    WA      0.0             0.0              0   
20776  2020-01-16    WA      0.0             0.0              0   
20777  2020-01-15    WA      0.0             0.0              0   
20778  2020-01-14    WA      0.0             0.0              0   
20779  2020-01-13    WA      0.0             0.0              0   

       deathProbable  hospitalized  hospitalizedCumulative  \
0                0.0        1293.0                  1293.0   
1             2185.0       45976.0                 45976.0   
2        

## Prepare the Langchain dataframe agent

In [16]:
agent = create_pandas_dataframe_agent(llm=model,df=df,verbose=True,allow_dangerous_code=True)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the DataFrame `df`, I can use the `shape` attribute which returns a tuple representing the dimensions of the DataFrame. The first element of this tuple is the number of rows.

Action: python_repl_ast
Action Input: df.shape[0]20780I now know the final answer
Final Answer: There are 20,780 rows in the dataframe `df`.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 20,780 rows in the dataframe `df`.'}

![](../images/21-db-agent.png)

## Design your prompt and ask your question

In [ ]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How may patients were hospitalized during July 2021" 
"in Texas, and nationwide as the total of all states?"
"Use the hospitalizedIncrease column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)

![](../images/22-db-agent.png)